In [234]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures


In [276]:
from sklearn.datasets import load_boston
import pandas as pd
df = pd.read_csv(load_boston()["filename"], skiprows=1)
print(load_boston()["DESCR"])
df.info()

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [145]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [255]:
df_init= df.copy()

In [270]:
def get_score(X,y, random_seed=42):
    model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed )  
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [271]:
def get_score_poly(X,y, deg =2, random_seed=42):
    model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed )  
    polynomial_features= PolynomialFeatures(degree=deg, include_bias=False)
    X_train_pl = polynomial_features.fit_transform(X_train)
    X_test_pl = polynomial_features.fit_transform(X_test)
    model.fit(X_train_pl, y_train)
    return model.score(X_test_pl, y_test)

In [272]:
def z_scaling(df, cols):
    for col in cols:
        df[col] -= df[col].mean()
        df[col] /= df[col].std()

In [283]:
columns = ["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT"]

Строю модель со всеми колонками

In [284]:
X = df.loc[:,columns]
y = df.loc[:,['MEDV']]
get_score(X,y)

0.7112260057484932

Маштабирую и строю модель

In [285]:
cols = ["CRIM","ZN","INDUS","NOX","RM","AGE","DIS","TAX","PTRATIO","B","LSTAT"]
z_scaling(df, cols)
X = df.loc[:,columns]
y = df.loc[:,['MEDV']]
get_score(X,y)

0.7112260057484932

Пробую обучить с добавлением полиноминальных признаков

In [280]:
get_score_poly(X, y,deg=2)

0.6610321968877226

Возможно проблема в том что много свойств но мало данных, пробую убрать параметры которые не улучшают предсказания

In [290]:
columns1 = ["CRIM","ZN","NOX","RM","DIS","RAD","TAX","PTRATIO","LSTAT"]
X = df.loc[:,columns1]
y = df.loc[:,['MEDV']]
get_score(X,y)

0.7208380046377809

In [295]:
df = df_init.copy()

добавляю новые свойство

In [317]:
df["CRRM"]= df.CRIM / df.RM
df["DISRAD"]=df["ZN"]*df["INDUS"]
df["RMLS"]=df.RM * df.LSTAT
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,CRRM,DISRAD,RMRAD,RMLS
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,0.000961,41.58,32.74350,32.74350
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,0.004253,0.00,58.68794,58.68794
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,0.003798,0.00,28.95555,28.95555
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,0.004626,0.00,20.57412,20.57412
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,0.009661,0.00,38.09351,38.09351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4,0.009499,0.00,63.75431,63.75431
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6,0.007397,0.00,55.56960,55.56960
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9,0.008710,0.00,39.34464,39.34464
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0,0.016130,0.00,44.02512,44.02512


In [318]:
columns = ["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT", "CRRM","DISRAD","RMLS"]
X = df.loc[:,columns]
y = df.loc[:,['MEDV']]
get_score(X,y)

0.8059310130255881